# Test Notebook

## Code

In [1]:
import random
import string
import subprocess
import os

In [2]:
os.chdir("..")

Read the dictionary

In [3]:
words = []
with open("data/american-english-sorted", 'r') as f:
    for word in f.readlines():
        words.append(word.replace('\n', ''))

Generate random prefixes

In [4]:
num_samples = 1000

In [5]:
prefixes = []
for word in random.sample(words, num_samples):
    while(len(word) < 2):
        word = random.choice(words)
    rand_int = random.randint(1, len(word)-1)
    prefix = word[:rand_int]
    prefixes.append(prefix)

Save prefix examples to file

In [6]:
os.makedirs("tmp", exist_ok=True)
with open("tmp/examples.txt", "w") as f:
    for x in prefixes:
        f.write(x)
        f.write("\n")

In [7]:
prefixes[:5]

['sh', 'runnies', 'co', 'Bra', 'fili']

To generate random prefixes with a fix length:

In [8]:
def get_random(prefix_len: int):
    while True:
        word = random.choice(words)
        if len(word) >= prefix_len:
            break
    return word[:prefix_len]

def get_result(prefix: str):
    return [x for x in words if x.startswith(prefix)]

In [9]:
prefix = get_random(prefix_len=4)
results = get_result(prefix)
print(prefix) 
print(results)

robo
['robocall', "robocall's", 'robocalled', 'robocalling', 'robocalls', 'robot', "robot's", 'robotic', 'robotics', "robotics's", 'robots']


In [10]:
def gen_results():
    os.chdir("bin")
    program_path = './autocomplete'
    arguments = ["../tmp/examples.txt"]
    subprocess.run([program_path] + arguments, text=True, capture_output=True)
    os.chdir("..")

def get_cpp_results():
    results = []
    with open("output_files/output_autocomplete.txt", 'r') as f:
        for line in f.readlines():
            line = line.replace("\n", "").strip()
            if line == ']':
                yield results
                results = []
            if line and not line == '[' and not line == ']':
                results.append(line)
            

def gen_expected_results():
    with open("tmp/examples.txt", 'r') as f:
        for line in f.readlines():
            line = line.replace('\n', '').strip()
            yield [x for x in words if x.startswith(line)]

def check_autocomplete():
    keys = []
    with open("tmp/examples.txt", 'r') as f:
        for line in f.readlines():
            line = line.replace('\n', '').strip()
            keys.append(line)
        
    for key, l1, l2 in zip(keys, get_cpp_results(), gen_expected_results()):
        if l1 != l2:
            print(f"Failed for {key}")
            print([x for x in l1 if x not in l2])
            print([x for x in l2 if x not in l1])
            return False
        else:
            print(f"Checked {key}")

In [11]:
gen_results()

In [12]:
check_autocomplete()

Checked sh
Checked runnies
Checked co
Checked Bra
Checked fili
Checked tr
Checked nitra
Checked ele
Checked carous
Checked Tahit
Checked Tr
Checked spur
Checked Salinas'
Checked condor'
Checked swi
Checked putrefie
Checked d
Checked N
Checked gam
Checked per
Checked g
Checked c
Checked overnight
Checked Sta
Checked D
Checked r
Checked ba
Checked f
Checked th
Checked Pyth
Checked a
Checked h
Checked d
Checked canoei
Checked cons
Checked Mo
Checked i
Checked scanda
Checked V
Checked abominat
Checked delphi
Checked in
Checked unde
Checked unc
Checked GNP
Checked tease
Checked Deer
Checked geare
Checked accr
Checked countr
Checked gladd
Checked disorderi
Checked folder'
Checked h
Checked Orient
Checked time
Checked recesse
Checked hook
Checked cowha
Checked windsoc
Checked sc
Checked reforma
Checked scamme
Checked precious
Checked ben
Checked comprehen
Checked g
Checked Araucani
Checked mo
Checked Sat
Checked forklif
Checked insignifican
Checked C
Checked wi
Checked quadr
Checked super
Che

# Binary Search

In [ ]:
def get_bin_search_results(text: str):
    os.chdir("bin")
    program_path = './binary_search'
    arguments = [text]
    subprocess.run([program_path] + arguments)
    with open("../output_files/output_bin_search.txt", 'r') as f:
        result = [x.replace('\n', '') for x in f.readlines()]
    os.chdir("..")
    return result

def check_bin_search(text):
    l1 = get_bin_search_results(text)
    l2 = [x for x in words if x.startswith(text)]
    return l1 == l2

In [ ]:
for prefix in prefixes:
    print(f"Checking {prefix}")
    if not check_bin_search(prefix):
        raise Exception(f"Failed for {prefix}")

# Levestein

In [ ]:
import textdistance

In [ ]:
retrieve_levestein_dist = textdistance.levenshtein

In [ ]:
def levestein(text, max_dist):
    output = []
    for word in words:
        dist = retrieve_levestein_dist(word, text)
        if dist <= max_dist:
            output.append(word)
    return output

def get_cpp_levestein_results(text: str, dist: int):
    os.chdir("bin")
    program_path = './levestein'
    arguments = [text, str(dist)]
    subprocess.run([program_path] + arguments, text=True, capture_output=True)
    with open("../output_files/output_levestein.txt", 'r') as f:
        result = [x.replace('\n', '') for x in f.readlines()]
    result.sort()
    os.chdir("..")
    return result

def check_levestein(text, dist):
    return get_cpp_levestein_results(text, dist) == levestein(text, dist)

In [ ]:
get_cpp_levestein_results("AAA", 1)

In [ ]:
levestein("AAA", 1)

In [ ]:
check_levestein("AAA", 1)

In [ ]:
# [delete , add, replace, maintain]
maintain = 0.9
others = (1 - maintain)/3
porcentages = [others, others, others, maintain]
assert sum(porcentages) == 1
accumulated = 0
intervals = []
for porcentage in porcentages:
    accumulated += porcentage
    intervals.append(accumulated)

In [ ]:
def get_action(number: float):
    output = 0
    while number > intervals[output]:
        output += 1
    return output

In [ ]:
num_samples = 10

In [ ]:
words_levestein = []
originals = []
for word in random.sample(words, num_samples):
    curr_word = ""
    curr_char_idx = 0
    while curr_char_idx != len(word):
        action = get_action(random.uniform(0, 1))
        # Delete char
        if action == 0:
            curr_char_idx += 1
        # Add random char
        elif action == 1:
            random_char = random.choice(string.ascii_letters)
            curr_word += random_char
        # Replace char
        elif action == 2:
            random_char = random.choice(string.ascii_letters)
            curr_word += random_char
            curr_char_idx += 1
        # Maintain char
        elif action == 3:
            curr_word += word[curr_char_idx]
            curr_char_idx += 1
    words_levestein.append(curr_word)
    originals.append(word)

In [ ]:
index = 1
words_levestein[index], originals[index]

In [ ]:
errors = []
for word in words_levestein[:100]:
    print(f"Checking {word}")
    for dist in range(4):
        if not check_levestein(word, dist):
            print(f"Failed for {word} and {dist}")
            errors.append((word, dist))